In [1]:
base_path = '/home/u5169119/PatchCL-MedSeg-jiyu'
dataset_path = '/home/u5169119/dataset/ACDC'

In [4]:
import sys
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import math
import time
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
sys.path.append(base_path)

from utils.transform import Transform
from utils.stochastic_approx import StochasticApprox
from utils.model import Network
from utils.datasets import LabData,UnlabData
from utils.queues import Embedding_Queues
from utils.CELOSS import CE_loss
from utils.patch_utils import _get_patches
from utils.aug_utils import batch_augment
from utils.get_embds import get_embeddings
from utils.const_reg import consistency_cost
from utils.plg_loss import PCGJCL
from utils.torch_poly_lr_decay import PolynomialLRDecay

In [5]:
contrastive_batch_size = 128
img_size = 256
batch_size = 8
num_classes = 4
transform = Transform(img_size, num_classes)
embd_queues = Embedding_Queues(num_classes)
stochastic_approx = StochasticApprox(num_classes,0.5,0.8)
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [6]:
model = Network()
teacher_model = Network()

#Turning off gradients for teacher model
for param in teacher_model.parameters():
    param.requires_grad=False
    #Esuring mothe the models have same weight
teacher_model.load_state_dict(model.state_dict())
model.contrast=False
teacher_model.contrast = False

model = nn.DataParallel(model)
model = model.to(dev)
teacher_model = nn.DataParallel(teacher_model)
teacher_model=teacher_model.to(dev)

In [7]:
cross_entropy_loss=CE_loss(num_classes, image_size=img_size)
metrics=[smp.utils.metrics.IoU(threshold=0.5)]

optimizer_pretrain=torch.optim.Adam(model.parameters(),lr=0.001)
optimizer_ssl=torch.optim.SGD(model.parameters(),lr=0.007)
scheduler = PolynomialLRDecay(optimizer=optimizer_pretrain, max_decay_steps=200, end_learning_rate=0.0001, power=2.0)

In [8]:
labeled_dataset = LabData(base_dir=dataset_path, split='train', transform=transform)
unlabeled_dataset = UnlabData(base_dir=dataset_path, split='train', transform=transform)
labelled_dataloader = DataLoader(labeled_dataset,batch_size=batch_size,shuffle=True)
unlabeled_dataloader = DataLoader(unlabeled_dataset,batch_size=batch_size,shuffle=True)


print('number of labeled_dataset: ', len(labeled_dataset))
print('number of unlabeled_dataset: ', len(unlabeled_dataloader))
print('=====================================')
for batch in labelled_dataloader:
    imgs = batch['image']
    masks = batch['label']
    
    # 將張量轉換為 numpy 數組以進行檢查
    images_np = imgs.numpy()
    
    # 檢查圖像的最小值和最大值
    min_val = np.min(images_np)
    max_val = np.max(images_np)
    
    print(f"Image min value: {min_val}")
    print(f"Image max value: {max_val}")
    
    break

print('image shape: ', imgs.shape)
print('masks shape: ', masks.shape)

# 每個 channel pixel 內容
for i in range(num_classes):
    mask_unique, mask_counts = np.unique(masks[1][i], return_counts=True)
    mask_pixel_counts = dict(zip(mask_unique, mask_counts))
    print(f'class {i} mask_pixel_counts: ', mask_pixel_counts)
    
    
plt.figure(figsize=(15, 15))
# 左邊顯示原圖
plt.subplot(3, 2, 1)
plt.imshow(imgs[0].permute(1, 2, 0))
plt.title("Labeled Dataset Original Image")

# 顯示各個通道的分割掩碼
plt.subplot(3, 2, 2)
plt.imshow(masks[0][0], cmap='gray')
plt.title("Segmentation Mask (Channel 0)")

plt.subplot(3, 2, 3)
plt.imshow(masks[0][1], cmap='gray')
plt.title("Segmentation Mask (Channel 1)")

plt.subplot(3, 2, 4)
plt.imshow(masks[0][2], cmap='gray')
plt.title("Segmentation Mask (Channel 2)")

plt.subplot(3, 2, 5)
plt.imshow(masks[0][3], cmap='gray')
plt.title("Segmentation Mask (Channel 3)")

plt.tight_layout()
plt.show()

print('=====================================')
for batch in unlabeled_dataloader:
    imgs = batch['image']
    break
print('image shape: ', imgs.shape)
# 左邊顯示原圖
plt.imshow(imgs[0].permute(1, 2, 0))
plt.title("Unlabeled Dataset Original Image")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/u5169119/dataset/ACDC/train_slices.list'

In [ ]:
#Send psudo masks & imgs to cpu
p_masks=masks
imgs = imgs

#get classwise patch list
patch_list = _get_patches(imgs, p_masks,classes=num_classes,background=True,img_size=img_size,patch_size=contrastive_batch_size)

#stochastic approximation filtering and threshold update
#qualified_patch_list = stochastic_approx.update(patch_list)
qualified_patch_list = patch_list

#make augmentations for teacher model
augmented_patch_list = batch_augment(qualified_patch_list,contrastive_batch_size)


#convert to tensor
aug_tensor_patch_list=[]
qualified_tensor_patch_list=[]
for i in range(len(augmented_patch_list)):
    if augmented_patch_list[i] is not None:
        aug_tensor_patch_list.append(torch.tensor(augmented_patch_list[i]))
        qualified_tensor_patch_list.append(torch.tensor(qualified_patch_list[i]))
    else:
        aug_tensor_patch_list.append(None)
        qualified_tensor_patch_list.append(None)

In [ ]:
#get embeddings of qualified patches through student model
model=model.train()
model.module.contrast=True
student_emb_list = get_embeddings(model,qualified_tensor_patch_list,True)
print('student_emb_list len: ', len(student_emb_list))


#get embeddings of augmented patches through teacher model
teacher_model.train()
teacher_model.module.contrast = True
teacher_embedding_list = get_embeddings(teacher_model,aug_tensor_patch_list,False)
print('teacher_embedding_list len: ', len(teacher_embedding_list))

#enqueue these
embd_queues.enqueue(teacher_embedding_list)

#calculate PCGJCL loss
PCGJCL_loss = PCGJCL(student_emb_list, embd_queues, 128, 0.2 , 4, psi=4096)
PCGJCL_loss = PCGJCL_loss.to(dev)
print('PCGJCL_loss: ', PCGJCL_loss)

model.module.contrast=False
#calculate supervied loss
imgs, masks =imgs.to(dev), masks.to(dev)
out = model(imgs)
print('masks shape: ', masks.shape)
print('output shape: ', out.shape)

supervised_loss = cross_entropy_loss(out,masks)
supervised_loss = supervised_loss.to(dev)

print('supervised_loss: ', supervised_loss)

#total loss
loss = supervised_loss + 0.5*PCGJCL_loss